# Testing the `evaluate` module

**Authorship:**
Adam Klie, *07/12/2022*<br>
**Last Updated:** *10/07/2022*
***
**Description:**
Notebook for testing out the `evaluate` module.

In [2]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
import pandas as pd
import eugene as eu
import matplotlib.pyplot as plt

Global seed set to 13
2022-10-07 10:43:20.346817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Fontconfig warning: ignoring UTF-8: not a valid region tag


In [3]:
sdata = eu.datasets.random1000()
eu.pp.ohe_seqs_sdata(sdata)
eu.pp.reverse_complement_seqs_sdata(sdata)
eu.pp.train_test_split_sdata(sdata)

One-hot encoding sequences:   0%|          | 0/1000 [00:00<?, ?it/s]

SeqData object modified:
	ohe_seqs: None -> 1000 ohe_seqs added
SeqData object modified:
	ohe_rev_seqs: None -> 1000 ohe_rev_seqs added
SeqData object modified:
    seqs_annot:
        + train_val


In [4]:
model = eu.models.DeepBind(input_len=66, output_dim=1)

In [6]:
def test_predictions(sdata, model):
    eu.evaluate.predictions(
        model,
        sdata,
        target_keys="activity_0",
        out_dir=f"../../_output",
        file_label="test",
    )
    saved_t = pd.read_csv(
        f"../../_output/ssDeepBind_regression/test_predictions.tsv", index_col=0, sep="\t"
    )
    assert np.allclose(
        saved_t["predictions_0"].values,
        sdata.seqs_annot.loc[saved_t.index]["activity_0_predictions"].values,
    )
test_predictions(sdata, model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


No transforms given, assuming just need to tensorize.


/Users/adamklie/miniconda3/envs/eugene/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Predicting: 0it [00:00, ?it/s]

In [9]:
def test_train_val_predictions(sdata, model):
    eu.evaluate.train_val_predictions(
        model, sdata=sdata, target_keys="activity_0", train_key="train_val", out_dir="../../_output/"
    )
    saved_t = pd.read_csv("../../_output/ssDeepBind_regression/train_predictions.tsv", index_col=0, sep="\t")
    assert np.allclose(
        saved_t["predictions_0"].values,
        sdata.seqs_annot.loc[saved_t.index]["activity_0_predictions"].values,
    )
    saved_v = pd.read_csv("../../_output/ssDeepBind_regression/val_predictions.tsv", index_col=0, sep="\t")
    assert np.allclose(
        saved_v["predictions_0"].values,
        sdata.seqs_annot.loc[saved_v.index]["activity_0_predictions"].values,
    )
test_train_val_predictions(sdata, model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


No transforms given, assuming just need to tensorize.
No transforms given, assuming just need to tensorize.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

---